In [70]:
import os
import json
import pandas as pd
import traceback
from langchain_core.runnables import RunnablePassthrough
from langchain_core.callbacks import CallbackManager

In [3]:
pip install langchain-community


Note: you may need to restart the kernel to use updated packages.


In [71]:
from langchain_community.chat_models import ChatOllama

In [72]:
llm = ChatOllama(model="llama2")


In [6]:
%pip install langchain
%pip install langchain-core
%pip install langchain-community



Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [73]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import StreamingStdOutCallbackHandler 
from langchain_core.callbacks import CallbackManager 
from langchain_community.llms import Ollama 


In [74]:
str_output_parser = StrOutputParser()

In [76]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        "answer": "string"
},
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        "answer": "string"
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "A": "string",
            "B": "string",
            "C": "string",
            "D": "string"
        },
        "answer": "string"
    }
}

In [77]:
mcq_prompt = PromptTemplate.from_template("""You are an expert MCQ maker. Given the above text, it is your job to

create a quiz of {number} multiple choice questions for {subject} students in {tone} tone.
                                          ### TEXT FOR QUIZ###
                                          {text}

Make sure the questions are not repeated and check all the questions to be conforming the text as we Make sure to format your response like RESPONSE JSON below and use it as a guide.

Ensure to make {number}) MCQs

### RESPONSE_JSON

{response_json}
                                          
"""
)                                          

In [78]:
quiz_chain = mcq_prompt |llm | str_output_parser

In [79]:
prompt = PromptTemplate.from_template("""You are an expert english grammer checker. Given the below questions for {subject} students,
                                       check if the answers are correct. If not, provide the correct answer.
                                      ###QUIZ DATA ###
                                      {input}
                                      Also, provide explanation for the correct answer in brief and in {tone} tone. Check the Complexity level.
                                      
                                      """)

In [80]:
correction_check = prompt | llm | str_output_parser

In [81]:
sequence_of_prompts = ( RunnablePassthrough.assign(input=quiz_chain)| correction_check)

In [82]:
file_path =r"C:\Users\aksha\mcgen\data.txt"

In [83]:
if os.path.exists(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        # Read the entire content of the file into the variable
        input_text = f.read()
    print(f"File '{file_path}' loaded successfully.")
else:
    # Handle the case where the file is missing
    print(f"Error: The file '{file_path}' was not found. Using a placeholder text.")
    input_text = "Default placeholder text about the Solar System."

File 'C:\Users\aksha\mcgen\data.txt' loaded successfully.


In [84]:
response_json_string = json.dumps(RESPONSE_JSON)

In [85]:
chain_input = {
    "text": input_text, # This now contains the content of data.txt
    "number": "3",
    "subject": "Physics",
    "tone": "academic",
    "response_json": response_json_string
}

In [ ]:
print("--- Starting Chain Invocation ---")
final_output = sequence_of_prompts.invoke(chain_input)
print("--- Chain Execution Complete ---")


print("\nFinal Output (Corrected and Explained Quiz):\n")
print(final_output)


--- Starting Chain Invocation ---


In [88]:
json.loads(final_output)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)